In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [2]:
default_config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 25
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "simulated_annealing": {
                "initial_temperature": 25,
                "cooling_rate": 0.1,
                "max_iteration": 250,
                "max_time": 30,
                "max_iteration_without_improvement": 25,
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "algorithm": "genetic_algorithm",
        "config": {
            "max_iteration": 250,
            "population_size": 25,
            "elitism_size": 2,
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 20,
                    "max_shift_threshold": 8,
                    "group_penalty": 0.25,
                    "shift_penalty": 0.75
                },
                "timeslot_conflict": {
                    "assistant_conflict_penalty": 1,
                    "group_conflict_penalty": 0.5
                },
            },
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": True,
                    "elitism": False,
                    "tournament_size": 10
                },
                "crossover": {
                    "single_point": False,
                    "two_point": False,
                    "uniform": True,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": True,
                    "mutation_probability": 0.05
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}




In [3]:
from scheduling_algorithm.data_parser import ModuleData
from scheduling_algorithm.algorithms import (
    GeneticAlgorithm,
    GeneticLocalSearch
)
from scheduling_algorithm.utils.solution_generator import SolutionGenerator
from scheduling_algorithm.factory import WeeklyFactory

In [4]:
modules = ModuleData.get_modules_by_semester(1)

In [5]:
# generator = SolutionGenerator.from_data(default_config)
weekly_generator = SolutionGenerator.from_data(default_config)

Loading configuration...
Configuration loaded successfully.
Creating FitnessManager with fitness functions: 
Fitness(name=GroupAssignmentCapacityFitness, max_threshold=3, conflict_penalty=1)
Fitness(name=AssistantDistributionFitness, max_group_threshold=20, max_shift_threshold=8, group_penalty=0.25, shift_penalty=0.75)
Fitness(name=TimeslotConflictFitness, assistant_conflict_penalty=1, group_conflict_penalty=0.5)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=TournamentSelection)]
Configuring crossover operator:  [Crossover(name=UniformCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation), Mutation(name=RandomMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]
Creating Genetic Algorithm Object from Configuration File
Population Size:  25
Max Iteration:  250


In [6]:
import cProfile as profile
import pstats
from pstats import SortKey

profiler = profile.Profile()
profiler.enable()
solution = weekly_generator.generate_solution_weekly_test()
profiler.disable()

Generating population for module 1 week 1
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stagnation Detected, Increasing Mutation Probability
Stag

In [9]:
from scheduling_algorithm.structure.chromosome import Chromosome


In [7]:
gene = []
for chromosome in solution[:72]:
    gene.append({"laboratory": chromosome[0], "module": chromosome[1], "chapter": chromosome[2], "group": chromosome[3], "assistant": chromosome[4], "time_slot": (chromosome[5], chromosome[6], chromosome[7])})

In [10]:
chromosome_copy = Chromosome(gene)

In [26]:
chromosome_copy.gene_data[1]

(1, 1, 2, 1, 4, 1.6633134e+09, 'Friday', 'Shift6')

In [11]:
chromosome_copy.gene_data

array([(1, 1, 1,  1, 3, 1.6633134e+09, 'Friday', 'Shift4'),
       (1, 1, 2,  1, 4, 1.6630542e+09, 'Tuesday', 'Shift6'),
       (1, 1, 1,  2, 3, 1.6652142e+09, 'Saturday', 'Shift3'),
       (1, 1, 2,  2, 5, 1.6633998e+09, 'Saturday', 'Shift3'),
       (1, 1, 1,  3, 3, 1.6633134e+09, 'Friday', 'Shift3'),
       (1, 1, 2,  3, 3, 1.6643502e+09, 'Wednesday', 'Shift6'),
       (1, 1, 1,  4, 2, 1.6631406e+09, 'Wednesday', 'Shift2'),
       (1, 1, 2,  4, 2, 1.6633134e+09, 'Friday', 'Shift3'),
       (1, 1, 1,  5, 5, 1.6637454e+09, 'Wednesday', 'Shift4'),
       (1, 1, 2,  5, 4, 1.6633134e+09, 'Friday', 'Shift1'),
       (1, 1, 1,  6, 4, 1.6633998e+09, 'Saturday', 'Shift1'),
       (1, 1, 2,  6, 3, 1.6631406e+09, 'Wednesday', 'Shift3'),
       (1, 1, 1,  7, 4, 1.6644366e+09, 'Thursday', 'Shift4'),
       (1, 1, 2,  7, 5, 1.6640046e+09, 'Saturday', 'Shift2'),
       (1, 1, 1,  8, 5, 1.6633998e+09, 'Saturday', 'Shift2'),
       (1, 1, 2,  8, 6, 1.6633134e+09, 'Friday', 'Shift1'),
       (1, 1, 1

In [27]:
weekly_generator.algorithm.fitness_manager(chromosome_copy)

21.0

In [7]:
stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)
stats.print_stats()

#profiler file
profiler.dump_stats('profiler2.prof')

         16765126 function calls (16720280 primitive calls) in 18.808 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    52208    3.982    0.000    4.546    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\timeslot_conflict.py:32(__call__)
    26104    3.957    0.000    5.630    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\assistant_distribution.py:20(calculate_penalty)
    26104    2.349    0.000    2.655    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\group_assignment_conflict.py:31(calculate_penalty)
    24000    2.125    0.000    2.125    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\operator\repair\time_slot_repair.py:25(<listcomp>)
  9363148    1.489    0.000    1.489    0.000 {method 'a

In [2]:
from scheduling_algorithm.factory import WeeklyFactory

In [3]:
population = WeeklyFactory(4,1).generate_population(1, 1)

In [11]:
population[0]['time_slot_date'][0]

12.0

In [12]:
a, b, c = (1, 2, 3)

In [14]:
b

2

In [19]:
timeslotconflict(population[0])

5

In [20]:
assistantdist(population[0])

41

In [9]:
groupassign(population[0])

NameError: name 'groupassign' is not defined

In [22]:
from collections import defaultdict, Counter
from scheduling_algorithm.structure.chromosome import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

import numpy as np

#Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
class AssistantDistributionFitness(BaseFitness):
    def __init__(self):
        super().__init__("AssistantDistributionFitness")
        self.max_group_threshold = 200
        self.max_shift_threshold = 50
        self.group_penalty = 1
        self.shift_penalty = 1
        
    def __str__(self):
        message = f"Fitness(name={self.name}, max_group_threshold={self.max_group_threshold}, max_shift_threshold={self.max_shift_threshold}, group_penalty={self.group_penalty}, shift_penalty={self.shift_penalty})"
        return message

    def calculate_penalty(self, modules, assistants, groups, timeslots):
        total_penalty = 0
        for assistant in np.unique(assistants):
            assistant_mask = assistants == assistant
            assistant_modules = modules[assistant_mask]
            assistant_groups = groups[assistant_mask]
            assistant_timeslots = timeslots[assistant_mask]
            
            group_counts = Counter(zip(assistant_modules, assistant_groups))
            shift_counts = Counter(zip(assistant_modules, assistant_timeslots))
            
            #Assuming that all the chromosomes have the same module. If we need to generate the schedule for all the module at once, we need to change the way we calculate the penalty
            group_penalty = max(0, (len(group_counts) - self.max_group_threshold) * self.group_penalty)
            shift_penalty = max(0, (len(shift_counts) - self.max_shift_threshold) * self.shift_penalty)
            total_penalty += group_penalty + shift_penalty
        return total_penalty

In [23]:
from collections import defaultdict
from scheduling_algorithm.structure.chromosome import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

class TimeslotConflict(BaseFitness):
    def __init__(self):
        """Fitness function to penalize conflicts in timeslot assignment. (e.g. a group or assistant is assigned to the same timeslot more than once)
        """
        super().__init__("TimeslotConflictFitness")
        self.assistant_conflict_penalty = 2
        self.group_conflict_penalty = 0.5
        
    def __str__(self):
        message = f"Fitness(name={self.name}, assistant_conflict_penalty={self.assistant_conflict_penalty}, group_conflict_penalty={self.group_conflict_penalty})"
        return message

    def __call__(self, timeslots, entity_ids, penalty):
        total_penalty = 0
        # combined_data = np.column_stack((timeslots, entity_ids))
        # unique_combinations, counts = np.unique(combined_data, return_counts=True, axis=0)
        combined_data = list(zip(timeslots, entity_ids))
        counts = Counter(combined_data).values()
        for count in counts:
            if count > 1:
                total_penalty += (count - 1) * penalty
        return total_penalty
    
    def calculate_penalty(self, assistants, groups, timeslots):
        assistant_penalty = self(timeslots, assistants, self.assistant_conflict_penalty)
        group_penalty = self(timeslots, groups, self.group_conflict_penalty)
        return assistant_penalty + group_penalty

In [24]:
from collections import defaultdict
from scheduling_algorithm.structure.chromosome import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

class GroupAssignmentCapacityFitness(BaseFitness):
    def __init__(self):
        """Calculate penalty for exceeding the maximum number of groups that can be assigned to a single time slot in lab
        """
        super().__init__("GroupAssignmentCapacityFitness")
        self.max_threshold = 3
        self.conflict_penalty = 1
        
    def __str__(self):
        message = f"Fitness(name={self.name}, max_threshold={self.max_threshold}, conflict_penalty={self.conflict_penalty})"
        return message

    def calculate_penalty(self, labs, modules, groups, timeslots):
        total_penalty = 0
        combined_data = np.column_stack((labs, modules, timeslots))
        # unique_combinations, counts = np.unique(combined_data, return_counts=True, axis=0)
        combined_data = list(zip(labs, modules, timeslots))
        counts = Counter(combined_data).values()
        for i, count in enumerate(counts):
            if count > self.max_threshold:
                excess = count - self.max_threshold
                total_penalty += excess * self.conflict_penalty
        return total_penalty

In [25]:
from typing import List
from scheduling_algorithm.structure import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

from collections import defaultdict

class FitnessManager:
    def __init__(self, fitness_functions: List[BaseFitness]):
        self.fitness_functions = fitness_functions

    def __call__(self, chromosome: Chromosome) -> int:
        """Calculate the fitness of a chromosome"""
        labs = chromosome["laboratory"]
        modules = chromosome["module"]
        chapters = chromosome["chapter"]
        timeslots = chromosome["time_slot"]
        groups = chromosome["group"]
        assistants = chromosome["assistant"]
            
        # Calculate total fitness
        total_fitness = 0
        for fitness_function in self.fitness_functions:
            if isinstance(fitness_function, GroupAssignmentCapacityFitness):
                total_fitness += fitness_function.calculate_penalty(labs, modules, groups, timeslots)
            elif isinstance(fitness_function, AssistantDistributionFitness):
                total_fitness += fitness_function.calculate_penalty(modules, assistants, groups, timeslots)
            elif isinstance(fitness_function, TimeslotConflict):
                total_fitness += fitness_function.calculate_penalty(assistants, groups, timeslots)

        return total_fitness

In [26]:
timeslotconflict = FitnessManager([TimeslotConflict()])
assistantdist = FitnessManager([AssistantDistributionFitness()])
groupassign = FitnessManager([GroupAssignmentCapacityFitness()])

In [27]:
timeslotconflict(solution)

225.0

In [28]:
assistantdist(solution)

0

In [29]:
groupassign(solution)

9

In [16]:
from scheduling_algorithm.data_parser import CommonData

In [18]:
CommonData.get_schedule(2,2)

{'Friday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': False},
 'Monday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Tuesday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Saturday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Thursday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Wednesday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False}}

In [19]:
CommonData.get_schedule(2,2)['Saturday']['Shift2']

True

In [8]:
import numpy as np
array = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
array2 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [14]:
import timeit as t
t.timeit('np.column_stack((array, array2))', globals=globals())

5.612801700001

In [15]:
t.timeit('np.concatenate((array.reshape(-1, 1), array2.reshape(-1, 1)), axis=1)', globals=globals())

2.5291220000071917